In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import *
from sklearn.metrics import roc_auc_score, log_loss
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import optuna

In [2]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

train = train.apply(lambda x:x.fillna(x.mode()[0]))
test = test.apply(lambda x:x.fillna(x.mode()[0]))

In [5]:
# from sklearn.preprocessing import QuantileTransformer

# numeric_cols = X.select_dtypes(include=[np.number]).columns
# qt = QuantileTransformer(output_distribution='normal', random_state=42)
# qt = QuantileTransformer(output_distribution='uniform', random_state=42)
# X[numeric_cols] = qt.fit_transform(X[numeric_cols])
# test[numeric_cols] = qt.fit_transform(test[numeric_cols])

# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
# test[numeric_cols] = scaler.fit_transform(test[numeric_cols])


### 컬럼 조작


In [4]:
non_imp_cols = ['난자 혼합 경과일',
 '기증자 정자와 혼합된 난자 수',
 '동결 배아 사용 여부',
 '여성 주 불임 원인',
 '불명확 불임 원인',
 '부부 부 불임 원인',
 '착상 전 유전 진단 사용 여부',
 '시술 유형',
 '남성 부 불임 원인',
 '기증 배아 사용 여부',
 '불임 원인 - 정자 운동성',
 '남성 주 불임 원인',
 '불임 원인 - 자궁내막증',
 '불임 원인 - 정자 농도',
 '여성 부 불임 원인',
 '대리모 여부',
 '불임 원인 - 정자 면역학적 요인',
 '저장된 신선 난자 수',
 '부부 주 불임 원인',
 '착상 전 유전 검사 사용 여부',
 '불임 원인 - 여성 요인',
 '불임 원인 - 자궁경부 문제',
 'PGD 시술 여부',
 'PGS 시술 여부',
 '난자 채취 경과일',
 '난자 해동 경과일',
 '불임 원인 - 정자 형태']

train.drop(columns=non_imp_cols, inplace=True)
test.drop(columns=non_imp_cols, inplace=True)

In [5]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [6]:
categorical_columns = [x for x in categorical_columns if x not in non_imp_cols]

print(len(categorical_columns))

25


In [7]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

In [8]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
import pickle
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm  # 진행바 라이브러리

# 'model.pkl' 파일에 저장된 모델 파라미터를 불러오기
with open('./best_params_cat_drop.pkl', 'rb') as f:
    params = pickle.load(f)

n_splits = 3
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# 3. 교차검증 수행 및 결과 저장
log_loss_scores = []
roc_scores = []
models = []  # 각 Fold별 모델 저장

for fold, (train_idx, val_idx) in enumerate(tqdm(skf.split(X_train, y_train), total=n_splits, desc="Fold 진행")):
    print(f"\n📌 Fold {fold+1}/{n_splits} 시작...")
    
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    # CatBoost 모델 설정
    model = CatBoostClassifier(
        **params,
        loss_function='Logloss',
        verbose=0,
        cat_features=categorical_columns,
        random_state=42
    )
    
    # 모델 학습
    model.fit(X_tr, y_tr, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=100)
    
    # 검증 데이터에 대한 예측 및 Log Loss 계산
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    loss = log_loss(y_val, y_pred_proba)
    roc = roc_auc_score(y_val, y_pred_proba)
    log_loss_scores.append(loss)
    roc_scores.append(roc)
    
    # 모델 저장
    models.append(model)
    print(f"✔ Fold {fold+1} Log Loss: {loss:.4f}")

print("\n평균 검증 ROC-AUC: {:.4f}".format(np.mean(roc_scores)))
print("\n평균 검증 Loss: {:.4f}".format(np.mean(log_loss_scores)))

# 4. 앙상블: 테스트 데이터에 대해 각 Fold 모델의 예측 확률 평균
ensemble_preds = np.zeros(X_valid.shape[0])
for model in models:
    ensemble_preds += model.predict_proba(X_valid)[:, 1]
ensemble_preds /= len(models)

# 5. 최종 테스트 데이터 ROC-AUC 평가
ensemble_auc = roc_auc_score(y_valid, ensemble_preds)
print("앙상블 모델의 테스트 ROC-AUC: {:.4f}".format(ensemble_auc))

"""
standard scaler
평균 검증 ROC-AUC: 0.7392

평균 검증 Loss: 0.4889

앙상블 모델의 테스트 ROC-AUC: 0.7390

qunatile transformer
평균 검증 ROC-AUC: 0.7389

평균 검증 Loss: 0.4892

앙상블 모델의 테스트 ROC-AUC: 0.7391
"""

Fold 진행:   0%|          | 0/3 [00:00<?, ?it/s]


📌 Fold 1/3 시작...
0:	learn: 0.6120678	test: 0.6123170	best: 0.6123170 (0)	total: 242ms	remaining: 1m 19s
100:	learn: 0.4850682	test: 0.4892985	best: 0.4892750 (98)	total: 9.97s	remaining: 22.6s
200:	learn: 0.4803915	test: 0.4891353	best: 0.4890280 (164)	total: 20.1s	remaining: 12.9s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.4890280042
bestIteration = 164

Shrink model to first 165 iterations.


Fold 진행:  33%|███▎      | 1/3 [00:23<00:47, 23.90s/it]

✔ Fold 1 Log Loss: 0.4890

📌 Fold 2/3 시작...
0:	learn: 0.6140371	test: 0.6137921	best: 0.6137921 (0)	total: 217ms	remaining: 1m 11s
100:	learn: 0.4845792	test: 0.4891695	best: 0.4891685 (99)	total: 9.99s	remaining: 22.6s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.4887539603
bestIteration = 131

Shrink model to first 132 iterations.


Fold 진행:  67%|██████▋   | 2/3 [00:44<00:22, 22.22s/it]

✔ Fold 2 Log Loss: 0.4888

📌 Fold 3/3 시작...
0:	learn: 0.6116527	test: 0.6118391	best: 0.6118391 (0)	total: 140ms	remaining: 46.2s
100:	learn: 0.4844820	test: 0.4902900	best: 0.4902900 (100)	total: 9.52s	remaining: 21.6s
200:	learn: 0.4790675	test: 0.4899879	best: 0.4898964 (193)	total: 20s	remaining: 12.9s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.4898963807
bestIteration = 193

Shrink model to first 194 iterations.


Fold 진행: 100%|██████████| 3/3 [01:12<00:00, 24.27s/it]

✔ Fold 3 Log Loss: 0.4899

평균 검증 ROC-AUC: 0.7389

평균 검증 Loss: 0.4892


앙상블 모델의 테스트 ROC-AUC: 0.7391


'\nstandard scaler\n평균 검증 ROC-AUC: 0.7392\n\n평균 검증 Loss: 0.4889\n\n앙상블 모델의 테스트 ROC-AUC: 0.7390\n\n\n'

In [10]:
ensemble_preds = np.zeros(test.shape[0])
for model in models:
    ensemble_preds += model.predict_proba(test)[:, 1]
ensemble_preds /= len(models)

print("예측 확률:", ensemble_preds)
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = ensemble_preds
sample_submission.to_csv('./submit/submit_26.csv', index=False)

예측 확률: [0.00818519 0.00298527 0.15506806 ... 0.43729072 0.23041865 0.00855836]


In [10]:
import pickle

# 'model.pkl' 파일에 저장된 모델 파라미터를 불러오기
with open('./best_params_cat_drop.pkl', 'rb') as f:
    params = pickle.load(f)

model = CatBoostClassifier(
        **params,
        loss_function='Logloss',
        verbose=0,
        cat_features=categorical_columns,
        random_state=42
)

model.fit(
        X_train, y_train,
        eval_set=(X_valid, y_valid),
        early_stopping_rounds=20
)

importances = model.get_feature_importance()
feature_names = X_train.columns
fi_df = pd.DataFrame({'feature': feature_names, 'importance': importances}).sort_values('importance', ascending=False)


predictions = model.predict_proba(X_valid)[:, 1]
print(roc_auc_score(y_valid, predictions))
# qt : 0.7392499423846555
# ss : 0.7393680351167298

0.7392141508678591


In [12]:
# 기본
# display(fi_df)
df_new = fi_df.loc[fi_df['importance'] < 0.1]
display(df_new['feature'].tolist())

['불임 원인 - 배란 장애',
 '불임 원인 - 난관 질환',
 '불임 원인 - 남성 요인',
 'DI 시술 횟수',
 '미세주입 후 저장된 배아 수',
 '신선 배아 사용 여부',
 '특정 시술 유형',
 '임신 시도 또는 마지막 임신 경과 연수',
 'DI 출산 횟수',
 '미세주입 배아 이식 수',
 '배아 해동 경과일',
 '해동 난자 수',
 '정자 출처',
 'DI 임신 횟수']

In [ ]:
# 기본
display(fi_df)
df_new = fi_df.loc[fi_df['importance'] != 0]
display(df_new)

feature  importance
41       이식된 배아 수   64.220291
1        시술 당시 나이    5.551073
43       저장된 배아 수    5.002989
38      총 생성 배아 수    3.835021
27    배아 생성 주요 이유    2.566004
..            ...         ...
60      PGD 시술 여부    0.000000
61      PGS 시술 여부    0.000000
62      난자 채취 경과일    0.000000
63      난자 해동 경과일    0.000000
26  불임 원인 - 정자 형태    0.000000

[67 rows x 2 columns]

feature  importance
41               이식된 배아 수   64.220291
1                시술 당시 나이    5.551073
43               저장된 배아 수    5.002989
38              총 생성 배아 수    3.835021
27            배아 생성 주요 이유    2.566004
47            수집된 신선 난자 수    2.040867
65              배아 이식 경과일    1.602149
5                배란 자극 여부    1.418348
49               혼합된 난자 수    1.357876
52                  난자 출처    1.081688
29          클리닉 내 총 시술 횟수    1.080656
50       파트너 정자와 혼합된 난자 수    1.061449
40        미세주입에서 생성된 배아 수    0.976512
6                배란 유도 유형    0.929871
32                총 임신 횟수    0.867586
36              IVF 출산 횟수    0.620854
19          불임 원인 - 배란 장애    0.461936
54              난자 기증자 나이    0.413622
0                시술 시기 코드    0.398551
42           미세주입 배아 이식 수    0.372221
30              IVF 시술 횟수    0.352867
2   임신 시도 또는 마지막 임신 경과 연수    0.296684
18          불임 원인 - 남성 요인    0.241837
39             미세주입된 난자 수    0.226033
55              정자 기증자 나이    0.223445
45               해동된 배아 수    0.209599
35                총 출산 횟수    0.200335
28                총 시술 횟수    0.178264
4                특정 시술 유형    0.174072
17          불임 원인 - 난관 질환    0.156726
31               DI 시술 횟수    0.155596
33              IVF 임신 횟수    0.150019
7             단일 배아 이식 여부    0.143334
46                해동 난자 수    0.142426
44        미세주입 후 저장된 배아 수    0.135967
34               DI 임신 횟수    0.133942
57            신선 배아 사용 여부    0.130643
53                  정자 출처    0.128716
66              배아 해동 경과일    0.117822
37               DI 출산 횟수    0.115847
64              난자 혼합 경과일    0.091021
51       기증자 정자와 혼합된 난자 수    0.089615
56            동결 배아 사용 여부    0.087530
12             여성 주 불임 원인    0.049938
16              불명확 불임 원인    0.038018
15             부부 부 불임 원인    0.030305
9        착상 전 유전 진단 사용 여부    0.026379
3                   시술 유형    0.024316
11             남성 부 불임 원인    0.023046
58            기증 배아 사용 여부    0.018769
25         불임 원인 - 정자 운동성    0.011044
10             남성 주 불임 원인    0.009364
22          불임 원인 - 자궁내막증    0.009051
23          불임 원인 - 정자 농도    0.007703
13             여성 부 불임 원인    0.007102
59                 대리모 여부    0.002853
24     불임 원인 - 정자 면역학적 요인    0.000206

In [10]:
# standard scaler
display(fi_df)
df_new = fi_df.loc[fi_df['importance'] != 0]
display(df_new)

feature  importance
41          이식된 배아 수   80.680642
1           시술 당시 나이    4.106672
38         총 생성 배아 수    2.839445
27       배아 생성 주요 이유    2.529968
65         배아 이식 경과일    1.778122
..               ...         ...
61         PGS 시술 여부    0.000000
62         난자 채취 경과일    0.000000
63         난자 해동 경과일    0.000000
20     불임 원인 - 여성 요인    0.000000
8   착상 전 유전 검사 사용 여부    0.000000

[67 rows x 2 columns]

feature  importance
41               이식된 배아 수   80.680642
1                시술 당시 나이    4.106672
38              총 생성 배아 수    2.839445
27            배아 생성 주요 이유    2.529968
65              배아 이식 경과일    1.778122
43               저장된 배아 수    1.494529
47            수집된 신선 난자 수    1.130083
52                  난자 출처    0.808021
6                배란 유도 유형    0.674208
54              난자 기증자 나이    0.494848
33              IVF 임신 횟수    0.284928
30              IVF 시술 횟수    0.266735
49               혼합된 난자 수    0.244050
44        미세주입 후 저장된 배아 수    0.219608
0                시술 시기 코드    0.216802
42           미세주입 배아 이식 수    0.171578
29          클리닉 내 총 시술 횟수    0.161028
45               해동된 배아 수    0.154199
50       파트너 정자와 혼합된 난자 수    0.144869
40        미세주입에서 생성된 배아 수    0.136315
36              IVF 출산 횟수    0.123989
32                총 임신 횟수    0.122941
39             미세주입된 난자 수    0.097831
35                총 출산 횟수    0.089768
53                  정자 출처    0.087225
7             단일 배아 이식 여부    0.071085
28                총 시술 횟수    0.068708
37               DI 출산 횟수    0.066740
2   임신 시도 또는 마지막 임신 경과 연수    0.066577
55              정자 기증자 나이    0.065851
56            동결 배아 사용 여부    0.065386
57            신선 배아 사용 여부    0.059704
17          불임 원인 - 난관 질환    0.056935
18          불임 원인 - 남성 요인    0.056648
48            저장된 신선 난자 수    0.052055
31               DI 시술 횟수    0.041014
51       기증자 정자와 혼합된 난자 수    0.037736
46                해동 난자 수    0.034768
5                배란 자극 여부    0.029486
64              난자 혼합 경과일    0.026637
4                특정 시술 유형    0.026327
19          불임 원인 - 배란 장애    0.019716
16              불명확 불임 원인    0.017342
66              배아 해동 경과일    0.013903
34               DI 임신 횟수    0.012693
9        착상 전 유전 진단 사용 여부    0.011559
15             부부 부 불임 원인    0.009222
25         불임 원인 - 정자 운동성    0.006122
10             남성 주 불임 원인    0.005635
22          불임 원인 - 자궁내막증    0.004441
58            기증 배아 사용 여부    0.004036
12             여성 주 불임 원인    0.003518
23          불임 원인 - 정자 농도    0.002190
13             여성 부 불임 원인    0.002070
11             남성 부 불임 원인    0.001414
14             부부 주 불임 원인    0.001281
59                 대리모 여부    0.000620
21        불임 원인 - 자궁경부 문제    0.000178

In [10]:
# quantile transform
display(fi_df)
df_new = fi_df.loc[fi_df['importance'] != 0]
display(df_new)

feature  importance
41       이식된 배아 수   71.225708
43       저장된 배아 수    6.336121
1        시술 당시 나이    5.858333
38      총 생성 배아 수    2.686949
65      배아 이식 경과일    1.946295
..            ...         ...
61      PGS 시술 여부    0.000000
62      난자 채취 경과일    0.000000
63      난자 해동 경과일    0.000000
23  불임 원인 - 정자 농도    0.000000
20  불임 원인 - 여성 요인    0.000000

[67 rows x 2 columns]

feature  importance
41               이식된 배아 수   71.225708
43               저장된 배아 수    6.336121
1                시술 당시 나이    5.858333
38              총 생성 배아 수    2.686949
65              배아 이식 경과일    1.946295
52                  난자 출처    1.381438
6                배란 유도 유형    1.257522
47            수집된 신선 난자 수    1.226122
27            배아 생성 주요 이유    0.923097
17          불임 원인 - 난관 질환    0.561220
50       파트너 정자와 혼합된 난자 수    0.531062
30              IVF 시술 횟수    0.521399
35                총 출산 횟수    0.488166
29          클리닉 내 총 시술 횟수    0.471397
49               혼합된 난자 수    0.385583
0                시술 시기 코드    0.364990
44        미세주입 후 저장된 배아 수    0.352017
42           미세주입 배아 이식 수    0.342872
54              난자 기증자 나이    0.290251
40        미세주입에서 생성된 배아 수    0.274751
32                총 임신 횟수    0.212349
28                총 시술 횟수    0.175524
55              정자 기증자 나이    0.173951
45               해동된 배아 수    0.155801
5                배란 자극 여부    0.155220
4                특정 시술 유형    0.153104
39             미세주입된 난자 수    0.133585
53                  정자 출처    0.132341
36              IVF 출산 횟수    0.131486
57            신선 배아 사용 여부    0.113485
7             단일 배아 이식 여부    0.112878
18          불임 원인 - 남성 요인    0.101663
34               DI 임신 횟수    0.099007
56            동결 배아 사용 여부    0.096150
33              IVF 임신 횟수    0.089161
37               DI 출산 횟수    0.074266
31               DI 시술 횟수    0.068697
19          불임 원인 - 배란 장애    0.057932
15             부부 부 불임 원인    0.049966
2   임신 시도 또는 마지막 임신 경과 연수    0.048549
64              난자 혼합 경과일    0.046365
46                해동 난자 수    0.043795
51       기증자 정자와 혼합된 난자 수    0.027153
9        착상 전 유전 진단 사용 여부    0.018773
10             남성 주 불임 원인    0.016814
22          불임 원인 - 자궁내막증    0.015333
3                   시술 유형    0.014785
16              불명확 불임 원인    0.013850
25         불임 원인 - 정자 운동성    0.010216
12             여성 주 불임 원인    0.010074
13             여성 부 불임 원인    0.008866
58            기증 배아 사용 여부    0.006946
14             부부 주 불임 원인    0.004525
66              배아 해동 경과일    0.001261
48            저장된 신선 난자 수    0.000606
24     불임 원인 - 정자 면역학적 요인    0.000229

In [10]:
# uniform transform
display(fi_df)
df_new = fi_df.loc[fi_df['importance'] != 0]
display(df_new)

feature  importance
41          이식된 배아 수   70.307673
1           시술 당시 나이    5.073480
38         총 생성 배아 수    3.009302
65         배아 이식 경과일    2.351401
43          저장된 배아 수    2.317679
..               ...         ...
61         PGS 시술 여부    0.000000
62         난자 채취 경과일    0.000000
63         난자 해동 경과일    0.000000
20     불임 원인 - 여성 요인    0.000000
8   착상 전 유전 검사 사용 여부    0.000000

[67 rows x 2 columns]

feature  importance
41               이식된 배아 수   70.307673
1                시술 당시 나이    5.073480
38              총 생성 배아 수    3.009302
65              배아 이식 경과일    2.351401
43               저장된 배아 수    2.317679
0                시술 시기 코드    1.383140
27            배아 생성 주요 이유    1.224158
29          클리닉 내 총 시술 횟수    1.163463
30              IVF 시술 횟수    1.048123
52                  난자 출처    0.947658
47            수집된 신선 난자 수    0.918633
6                배란 유도 유형    0.876411
35                총 출산 횟수    0.721541
54              난자 기증자 나이    0.653320
40        미세주입에서 생성된 배아 수    0.586758
33              IVF 임신 횟수    0.575318
5                배란 자극 여부    0.541395
42           미세주입 배아 이식 수    0.511128
50       파트너 정자와 혼합된 난자 수    0.495243
45               해동된 배아 수    0.465868
36              IVF 출산 횟수    0.429172
53                  정자 출처    0.390980
44        미세주입 후 저장된 배아 수    0.380865
4                특정 시술 유형    0.291187
39             미세주입된 난자 수    0.278754
28                총 시술 횟수    0.274632
18          불임 원인 - 남성 요인    0.249678
32                총 임신 횟수    0.246710
57            신선 배아 사용 여부    0.239059
17          불임 원인 - 난관 질환    0.217896
7             단일 배아 이식 여부    0.207854
55              정자 기증자 나이    0.196679
49               혼합된 난자 수    0.193944
2   임신 시도 또는 마지막 임신 경과 연수    0.167648
37               DI 출산 횟수    0.145387
31               DI 시술 횟수    0.101680
34               DI 임신 횟수    0.087851
56            동결 배아 사용 여부    0.085127
16              불명확 불임 원인    0.079227
51       기증자 정자와 혼합된 난자 수    0.072192
46                해동 난자 수    0.071027
64              난자 혼합 경과일    0.070379
9        착상 전 유전 진단 사용 여부    0.058952
19          불임 원인 - 배란 장애    0.056145
11             남성 부 불임 원인    0.040975
14             부부 주 불임 원인    0.038290
13             여성 부 불임 원인    0.026794
48            저장된 신선 난자 수    0.026311
15             부부 부 불임 원인    0.022786
12             여성 주 불임 원인    0.021546
58            기증 배아 사용 여부    0.018980
10             남성 주 불임 원인    0.013961
25         불임 원인 - 정자 운동성    0.008900
66              배아 해동 경과일    0.007671
22          불임 원인 - 자궁내막증    0.004551
23          불임 원인 - 정자 농도    0.002912
21        불임 원인 - 자궁경부 문제    0.001583
24     불임 원인 - 정자 면역학적 요인    0.000024